# 1. Introductory Tutorial of Python’s SQLAlchemy
http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/

### Python's SQLAlchemy and Declarative

There are three most important components in writing SQLAlchemy code:
- A Table that represents a table in a database.
- A mapper that maps a Python class to a table in a database.  
- A class object that defines how a database record maps to a normal Python object. 

** tables are mapped as classes, rows as instances and columns as attributes.  **  

Instead of having to write code for Table, mapper and the class object at different places, SQLAlchemy's declarative allows a - Table, a mapper and a class object to be defined at once in one class definition.

The following declarative definitions specify the same tables defined in sqlite_ex.py:

In [31]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
 
Base = declarative_base()

In [32]:
class Person(Base):
    __tablename__ = 'person'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)

class Address(Base):
    __tablename__ = 'address'
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

# Create an engine that stores data in the local directory's sqlalchemy_example.db file.
engine = create_engine('sqlite:///sqlalchemy_example.db')
 
# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

Now a new sqlite3 db file called "sqlalchemy_example.db" should be created in your current directory.  
Since the sqlalchemy db is empty right now, let's write some code to insert records into the database:

In [35]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///sqlalchemy_example.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind = engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()
 
# Insert a Person in the person table
new_person = Person(name='new person')
session.add(new_person)
session.commit()
 
# Insert an Address in the address table
new_address = Address(post_code='00000', person=new_person)
session.add(new_address)
session.commit()

Save the previous code into a local file sqlalchemy_insert.py and run the command python sqlalchemy_insert.py in your shell.  
Now we have one Person object and one Address object stored in the database.  
Let's **query** the database using the classes defined in sqlalchemy_declarative.py:

In [41]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine
from sqlalchemy.orm import sessionmaker
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

In [45]:
# Make a query to find all Persons in the database
session.query(Person).all()

[<__main__.Person at 0x86b4828>, <__main__.Person at 0x86b4748>]

In [37]:
# Return the first Person from all Persons in the database
person = session.query(Person).first()
person.name

'new person'

In [51]:
# Find all Address whose person field is pointing to the person object
session.query(Address).filter(Address.person == person).all()

In [52]:
# Retrieve one Address whose person field is point to the person object
session.query(Address).filter(Address.person == person).one()

In [53]:
address = session.query(Address).filter(Address.person == person).one()
address.post_code

'00000'

### Summary of Python's SQLAlchemy

In this article, we learned how to write database code using SQLAlchemy's declaratives. Compared to writing the traditional raw SQL statements using sqlite3, SQLAlchemy's code is more object-oriented and easier to read and maintain. In addition, we can easily create, read, update and delete SQLAlchemy objects like they're normal Python objects.

You might be wondering that if SQLAlchemy's just a thin layer of abstraction above the raw SQL statements, then it's not very impressive and you might prefer to writing raw SQL statements instead. In the following articles of this series, we're going to investigate various aspects of SQLAlchemy and compare it against raw SQL statements when they're both used to implement the same functionalities. I believe at the end of this series, you will be convinced that SQLAlchemy is superior to writing raw SQL statements.

In [54]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
